In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [3]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,1112407232,1QP36VO,0.0,Part-time,Senior / Semi-Senior,Recursos Humanos,Consultores de Empresas SRL,146.0,240.0,85.0,7.0,FEM,31.196319,22.0,FEM,11
1,1112377175,1QP36VO,0.0,Full-time,Senior / Semi-Senior,Administración,Excelencia Laboral S.A.,69.0,87.0,36.0,2.0,FEM,30.991935,22.0,FEM,11
2,1112141215,1QP36VO,0.0,Full-time,Senior / Semi-Senior,Tesorería,Manpower,21.0,2722.0,1363.0,59.0,FEM,28.014962,22.0,FEM,11
3,1112258963,1QP36VO,0.0,Full-time,Senior / Semi-Senior,Gastronomia,BAYTON,24.0,11.0,62.0,4.0,MASC,33.369863,22.0,FEM,11
4,1112275246,1QP36VO,0.0,Full-time,Senior / Semi-Senior,Comercial,Adecco -Región Office,167.0,1423.0,1163.0,14.0,NaN,30.800774,22.0,FEM,11


In [4]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,42.0,FEM,21,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,31.0,MASC,9,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,36.0,FEM,12,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,69.0,MASC,15,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,32.0,FEM,11,1


In [5]:
avisos = None
postulantes = None

In [6]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante','id'],axis=1,inplace=True)

In [7]:
test.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,42.0,FEM,21
1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,31.0,MASC,9
2,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,36.0,FEM,12
3,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,69.0,MASC,15
4,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,32.0,FEM,11


In [8]:
postulaciones.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,Part-time,Senior / Semi-Senior,Recursos Humanos,Consultores de Empresas SRL,146.0,240.0,85.0,7.0,FEM,31.196319,22.0,FEM,11
1,Full-time,Senior / Semi-Senior,Administración,Excelencia Laboral S.A.,69.0,87.0,36.0,2.0,FEM,30.991935,22.0,FEM,11
2,Full-time,Senior / Semi-Senior,Tesorería,Manpower,21.0,2722.0,1363.0,59.0,FEM,28.014962,22.0,FEM,11
3,Full-time,Senior / Semi-Senior,Gastronomia,BAYTON,24.0,11.0,62.0,4.0,MASC,33.369863,22.0,FEM,11
4,Full-time,Senior / Semi-Senior,Comercial,Adecco -Región Office,167.0,1423.0,1163.0,14.0,NaN,30.800774,22.0,FEM,11


In [9]:
test = pd.concat([test,postulaciones])

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not test[col].dtype == 'O': continue
    print(col)
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])
test.head()

tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
TendenciaSexo
sexo


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,1,1,30,448,102.0,NaN,NaN,NaN,2,NaN,42.0,1,21
1,1,1,30,448,102.0,NaN,NaN,NaN,2,NaN,31.0,2,9
2,1,1,30,448,102.0,NaN,NaN,NaN,2,NaN,36.0,1,12
3,1,3,170,3810,27.0,NaN,NaN,NaN,2,NaN,69.0,2,15
4,1,3,170,3810,27.0,NaN,NaN,NaN,2,NaN,32.0,1,11


In [11]:
test = test.fillna(-1)
postulaciones = test[100000:]
test = test[:100000]

In [12]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [13]:
postulaciones.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2,4,154,954,146.0,240.0,85.0,7.0,0,31.196319,22.0,1,11
1,1,4,2,1497,69.0,87.0,36.0,2.0,0,30.991935,22.0,1,11
2,1,4,175,2570,21.0,2722.0,1363.0,59.0,0,28.014962,22.0,1,11
3,1,4,82,389,24.0,11.0,62.0,4.0,1,33.369863,22.0,1,11
4,1,4,30,206,167.0,1423.0,1163.0,14.0,2,30.800774,22.0,1,11


In [14]:
test.head()


,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,1,30,448,102.0,-1.0,-1.0,-1.0,2,-1.0,42.0,1,21
1,1,1,30,448,102.0,-1.0,-1.0,-1.0,2,-1.0,31.0,2,9
2,1,1,30,448,102.0,-1.0,-1.0,-1.0,2,-1.0,36.0,1,12
3,1,3,170,3810,27.0,-1.0,-1.0,-1.0,2,-1.0,69.0,2,15
4,1,3,170,3810,27.0,-1.0,-1.0,-1.0,2,-1.0,32.0,1,11


In [48]:
import neurolab as nl